# GBLUP approach


## Get kinship matrix in the correct format for ASReml GBLUP

ASReml-R requires the genomic relationship matrix (G) to be inverted and in sparse format for computational efficiency in GBLUP models.

After calculating the genomic relationship matrix from SNP data, the resulting G matrix often has mathematical properties that can cause problems in mixed model equations:

- **Negative eigenvalues**: Some eigenvalues may be slightly negative due to sampling or numerical precision
- **Near-zero eigenvalues**: Small eigenvalues make matrix inversion unstable and can cause convergence failures

**Bending** addresses these issues by:
1. Performing eigenvalue decomposition of the G matrix
2. Adjusting problematic eigenvalues (those below a tolerance threshold) to ensure the matrix is positive definite
3. Reconstructing a "bent" matrix that is numerically stable for inversion

The default tolerance (`eig.tol = 1e-06`) sets the minimum acceptable eigenvalue.

In [ ]:
# Load library
library(ASRgenomics)  # Genomic relationship matrix processing

# Load previously created kinship matrix
load("kinship_matrix_VanRaden_auspak_maxmissing20.RData")

# Apply bending to ensure G matrix is positive definite
# eig.tol = 1e-06 is the default tolerance for minimum eigenvalue
G_bending <- G.tuneup(G = kinship_matrix_V2, bend = TRUE)

# Compute inverse of bent G matrix in sparse format for ASReml
# Sparse format reduces memory usage and improves computational speed
Ginv_sparse <- G.inverse(G_bending$Gb, sparse = TRUE)$Ginv

## load phenotypic data

In [ ]:
# Load libraries
library(dplyr)    # Data manipulation for CV workflow
library(asreml)   # GBLUP modeling


# load phenotype data
pheno_data <- read.csv("../data/AUSPAK_phenotypes_means_BLUEs.csv")

# year, SampleName, location as factors
pheno_data$year <- as.factor(pheno_data$year)
pheno_data$SampleName <- as.factor(pheno_data$SampleName)
pheno_data$location <- as.factor(pheno_data$location)
pheno_data$environment <- as.factor(pheno_data$environment)

# rename SampleName to sample.id
pheno_data <- pheno_data %>%
  rename(sample.id = SampleName)

## GBLUP using ASReml-R

In [ ]:
# GBLUP cross-validation with z-score transformation and NDCG evaluation

# function to calculate NDCG (Normalized Discounted Cumulative Gain)
calculate_ndcg <- function(y_true, y_pred, k = 10, lower_is_better = FALSE) {
  k <- min(k, length(y_true))
  
  # Flip values if lower is better
  if(lower_is_better) {
    y_true <- -y_true
    y_pred <- -y_pred
  }
  
  # Transform to non-negative values
  min_val <- min(c(y_true, y_pred))
  if(min_val < 0) {
    y_true_pos <- y_true - min_val + 1e-6
    y_pred_pos <- y_pred - min_val + 1e-6
  } else {
    y_true_pos <- y_true
    y_pred_pos <- y_pred
  }
  
  # Get ranking based on predicted values
  pred_order <- order(y_pred_pos, decreasing = TRUE)
  relevance <- y_true_pos[pred_order[1:k]]
  
  # Calculate DCG
  dcg <- 0
  for(i in 1:k) {
    dcg <- dcg + relevance[i] / log2(i + 1)
  }
  
  # Calculate IDCG
  ideal_relevance <- sort(y_true_pos, decreasing = TRUE)[1:k]
  idcg <- 0
  for(i in 1:k) {
    idcg <- idcg + ideal_relevance[i] / log2(i + 1)
  }
  
  if(idcg == 0) return(0)
  return(dcg / idcg)
}

# Evaluate predictions using Pearson correlation and NDCG
evaluate_predictions <- function(y_true, y_pred, trait_name = NULL) {
  if(length(y_true) < 2) {
    return(list(pearson = NA, ndcg_at_10 = NA))
  }
  
  # Define traits where lower values are better
  lower_is_better_traits <- c('DTF_blue', 'DTH_blue', 'PtHt_blue', 'DTF_mean', 'DTH_mean', 'PtHt_mean')
  lower_is_better <- if(!is.null(trait_name)) trait_name %in% lower_is_better_traits else FALSE
  
  tryCatch({
    list(
      pearson = cor(y_true, y_pred, use = "complete.obs"),
      ndcg_at_10 = calculate_ndcg(y_true, y_pred, k = 10, lower_is_better = lower_is_better)
    )
  }, error = function(e) {
    list(pearson = NA, ndcg_at_10 = NA)
  })
}

# Apply z-score transformation by environment
apply_environment_scaling <- function(pheno_data, traits) {
  scaled_data <- pheno_data
  
  for(trait in traits) {
    trait_data <- pheno_data %>% filter(!is.na(.data[[trait]]))
    if(nrow(trait_data) < 50) next
    
    # Calculate environment-specific statistics
    env_stats <- trait_data %>%
      group_by(environment) %>%
      summarise(
        env_mean = mean(.data[[trait]], na.rm = TRUE),
        env_sd = sd(.data[[trait]], na.rm = TRUE),
        .groups = 'drop'
      ) %>%
      mutate(env_sd = ifelse(env_sd == 0 | is.na(env_sd), 1, env_sd)) 
    
    # Apply transformation
    scaled_data <- scaled_data %>%
      left_join(env_stats, by = "environment") %>%
      mutate(
        !!trait := ifelse(!is.na(.data[[trait]]), 
                         (.data[[trait]] - env_mean) / env_sd, 
                         NA)
      ) %>%
      select(-env_mean, -env_sd)
  }
  
  return(scaled_data)
}

### Main function for GBLUP with group k-fold cross-validation and z-score transformation
group_kfold_gblup_zscore <- function(pheno_data, Ginv_sparse, traits, 
                                      k_folds = 10, n_iterations = 15, 
                                      apply_zscore = TRUE) {
  
  cat("Preparing data...\n")
  
  # Ensure proper factor levels
  pheno_data$sample.id <- as.factor(as.character(pheno_data$sample.id))
  pheno_data$environment <- as.factor(as.character(pheno_data$environment))
  pheno_data$location <- as.factor(as.character(pheno_data$location))
  pheno_data$year <- as.factor(as.character(pheno_data$year))
  
  # Validate G matrix for genotype presence and order
  geno_levels <- levels(pheno_data$sample.id)
  G_rownames <- rownames(Ginv_sparse)
  if(is.null(G_rownames)) G_rownames <- attr(Ginv_sparse, "rowNames")

  if(!all(geno_levels %in% G_rownames)) {
    stop("Not all genotypes in data are present in G matrix")
  }

  if(!identical(geno_levels, G_rownames)) {
    cat("Reordering genotype levels to match G matrix...\n")
    pheno_data$sample.id <- factor(pheno_data$sample.id, levels = G_rownames)
    if(!identical(levels(pheno_data$sample.id), G_rownames)) {
      stop("Failed to reorder genotype levels")
    }
  }
  
  cat("Data: ", length(geno_levels), "genotypes, ", 
      length(levels(pheno_data$environment)), "environments, ",
      nrow(pheno_data), "observations\n")
  cat("Z-score:", ifelse(apply_zscore, "ON", "OFF"), "\n\n")
  
  # Apply environment scaling if requested
  if(apply_zscore) {
    pheno_data <- apply_environment_scaling(pheno_data, traits)
  }
  
  # Initialize results dataframe
  cv_results <- data.frame(
    iteration = integer(),
    fold = integer(),
    trait = character(),
    location = character(),
    pearson = numeric(),
    ndcg_at_10 = numeric(),
    seed = integer(),
    n_test_genotypes = integer(),
    stringsAsFactors = FALSE
  )
  
  genotypes <- levels(pheno_data$sample.id)
  n_genotypes <- length(genotypes)
  
  cat("Starting CV:", n_genotypes, "genotypes |", k_folds, "folds |", n_iterations, "iterations\n\n")
  
  # Loop through iterations
  for(iter in 1:n_iterations) {
    current_seed <- 1000 + iter
    set.seed(current_seed)
    
    cat("Iteration", iter, "of", n_iterations, "(seed:", current_seed, ")\n")
    
    # Create genotype folds
    fold_assignments <- sample(rep(1:k_folds, length.out = n_genotypes))
    names(fold_assignments) <- genotypes
    
    # Loop through folds
    for(fold in 1:k_folds) {
      cat("  Fold", fold, "of", k_folds, "\n")
      
      # Get test genotypes for this fold
      test_genotypes <- names(fold_assignments)[fold_assignments == fold]
      
      # Loop through traits
      for(trait in traits) {
        tryCatch({
          
          # Filter data for environments with trait data
          envs_with_data <- pheno_data %>%
            filter(!is.na(.data[[trait]])) %>%
            distinct(environment) %>%
            pull(environment)
          
          if(length(envs_with_data) < 1) {
            warning(paste("No environments have data for trait", trait))
            next
          }
          
          # Filter dataset
          trait_data <- pheno_data %>%
            filter(environment %in% envs_with_data)
          trait_data$environment <- droplevels(trait_data$environment)
          
          # Create training data by excluding test genotypes
          train_data <- trait_data %>%
            filter(!sample.id %in% test_genotypes)
          
          # Check sufficient training data
          non_na_count <- sum(!is.na(train_data[[trait]]))
          if(non_na_count < 50) {
            warning(paste("Insufficient training data for", trait))
            next
          }
          
          # Fit GBLUP model
          model <- asreml(
            fixed = as.formula(paste(trait, "~ location")),
            random = ~ vm(sample.id, Ginv_sparse) + location:year, 
            residual = ~ units,
            na.action = na.method(y = "include"),
            workspace = 256e06,
            data = train_data,
            maxit = 200,
            trace = FALSE
          )
          
          # Check model convergence
          if(is.null(model) || !model$converge) {
            warning(paste("Model convergence failed for", trait))
            next
          }
          
          # Get predictions for test genotypes
          pred_data <- trait_data[trait_data$sample.id %in% test_genotypes, ]
          pred_data[[trait]] <- NA
          
          predictions <- predict(model, classify = "sample.id:location:year",
                                newdata = pred_data, pworkspace = 256e06, trace = FALSE)
          
          if(is.null(predictions$pvals)) {
            warning(paste("Could not extract predictions for trait", trait))
            next
          }
          
          pred_values <- predictions$pvals
          pred_values$environment <- paste(pred_values$location, pred_values$year, sep = "_")
          
          # Calculate results by location
          results_by_location <- trait_data %>%
            filter(sample.id %in% test_genotypes & !is.na(.data[[trait]])) %>%
            group_by(sample.id, location) %>%
            summarise(observed_mean = mean(.data[[trait]], na.rm = TRUE), .groups = 'drop') %>%
            left_join(
              pred_values %>%
                filter(sample.id %in% test_genotypes) %>%
                group_by(sample.id, location) %>%
                summarise(predicted_mean = mean(predicted.value, na.rm = TRUE), .groups = 'drop'),
              by = c("sample.id", "location")
            ) %>%
            filter(!is.na(predicted_mean))
          
          # Evaluate by location
          unique_locations <- unique(results_by_location$location)
          
          for(loc in unique_locations) {
            loc_data <- results_by_location %>% filter(location == loc)
            
            if(nrow(loc_data) >= 3) {
              eval_results <- evaluate_predictions(loc_data$observed_mean, loc_data$predicted_mean, trait_name = trait)
              
              if(!is.na(eval_results$pearson)) {
                cv_results <- rbind(cv_results, data.frame(
                  iteration = iter,
                  fold = fold,
                  trait = trait,
                  location = as.character(loc),
                  pearson = eval_results$pearson,
                  ndcg_at_10 = eval_results$ndcg_at_10,
                  seed = current_seed,
                  n_test_genotypes = length(unique(loc_data$sample.id)),
                  stringsAsFactors = FALSE
                ))
                
                cat("    ", trait, "- Location:", loc, 
                    "- Pearson:", round(eval_results$pearson, 3),
                    "- NDCG@10:", round(eval_results$ndcg_at_10, 3),
                    "| N genotypes:", nrow(loc_data), "\n")
              }
            } else {
              cat("    ", trait, "- Location:", loc, "- Insufficient data (N=", nrow(loc_data), ")\n")
            }
          }
          
        }, error = function(e) {
          warning(paste("Error in trait", trait, ":", e$message))
        })
      }
    }
  }
  
  # Calculate summary statistics
  if(nrow(cv_results) > 0) {
    summary_stats <- cv_results %>%
      group_by(trait, location) %>%
      summarise(
        across(c(pearson, ndcg_at_10), 
               list(mean = mean, sd = sd, min = min, max = max), 
               na.rm = TRUE),
        n_validations = n(),
        mean_n_test_genotypes = mean(n_test_genotypes, na.rm = TRUE),
        .groups = 'drop'
      )
    
    cat("\n=== Cross-Validation Summary ===\n")
    print(summary_stats[, c('trait', 'location', 'pearson_mean', 'ndcg_at_10_mean')])
  } else {
    warning("No successful cross-validation results obtained")
    summary_stats <- data.frame()
  }
  
  return(list(
    results = cv_results,
    summary = summary_stats,
    n_genotypes = n_genotypes,
    n_environments = length(levels(pheno_data$environment)),
    zscore_applied = apply_zscore
  ))
}

In [ ]:
traits <- c('PtHt_mean', 'PcleLng_mean', 'TGW_mean', 'SdLen_mean', 'DTF_mean', 'DTH_mean', 'PtHt_blue', 'PcleLng_blue', 'SdW_z_mean', 'SdW_z_blue', 'TGW_blue', 'SdLen_blue', 'DTF_blue', 'DTH_blue')

cv_output_zscore <- group_kfold_gblup_zscore(
  pheno_data = pheno_data,
  Ginv_sparse = Ginv_sparse,
  traits = traits,
  k_folds = 5,
  n_iterations = 15, 
  apply_zscore = TRUE
)

In [ ]:
traits <- c('PtHt_mean', 'PcleLng_mean', 'TGW_mean', 'SdLen_mean', 'DTF_mean', 'DTH_mean', 'PtHt_blue', 'PcleLng_blue', 'SdW_z_mean', 'SdW_z_blue', 'TGW_blue', 'SdLen_blue', 'DTF_blue', 'DTH_blue')

cv_output_nozscore <- group_kfold_gblup_zscore(
  pheno_data = pheno_data,
  Ginv_sparse = Ginv_sparse,
  traits = traits,
  k_folds = 5,
  n_iterations = 15, 
  apply_zscore = FALSE
)